In [ ]:
import json, os, glob
from PIL import Image, ImageDraw
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

# Define label mappings for segmentation classes
class_mappings = {'NFL': 13, 'GCL+IPL': 5, 'INL': 17, 'OPL': 2, 'ONL': 22, 'Elipsoid Zone': 9, 'RPE': 3, 'Choroid': 31}

# Load JSON annotations
annotations_file = '/home/khamituly/Documents/Diploma work/labels_8_layer_2022-06-08-03-14-52.json'
with open(annotations_file) as file:
    annotations = json.load(file)

# Directory paths
input_images_dir = '/home/khamituly/Documents/Diploma work/dataset_manual/temp_health/X/'
output_masks_dir = '/home/khamituly/Documents/Diploma work/dataset_manual/temp_health/Y_test/'

# Function to extract polygon data from JSON
def extract_polygon_data(data):
    x_coords, y_coords, labels = [], [], []
    for region in data['regions']:
        x_coords.append(data['regions'][region]['shape_attributes']['all_points_x'])
        y_coords.append(data['regions'][region]['shape_attributes']['all_points_y'])
        labels.append(data['regions'][region]['region_attributes']['label'])
    return x_coords, y_coords, labels

# Create mask using polygon data
def generate_mask(x_coords, y_coords, width, height, labels, label_mapping):
    mask = np.zeros((width, height))
    for xs, ys, label in zip(x_coords, y_coords, labels):
        poly_coords = np.vstack((xs, ys)).reshape((-1,), order='F').tolist()
        img = Image.new("L", (width, height), 0)
        ImageDraw.Draw(img).polygon(poly_coords, fill=1, outline=1)
        poly_mask = np.flipud(np.rot90(np.array(img)))

        if label in label_mapping:
            mask[poly_mask == 1] = label_mapping[label]
    return mask

# Rescale function to normalize pixel values
def normalize(data, min_val, max_val):
    data_min, data_max = data.min(), data.max()
    return (max_val - min_val) * (data - data_min) / (data_max - data_min) + min_val

# Process each image and generate corresponding mask
for image_name, annotation_data in annotations.items():
    image_path = os.path.join(input_images_dir, image_name)
    image = Image.open(image_path)
    width, height = image.size
    x_coords, y_coords, labels = extract_polygon_data(annotation_data)
    mask = generate_mask(x_coords, y_coords, width, height, labels, class_mappings)

    mask_image = Image.fromarray(normalize(mask, 0, 255)).convert('RGB')
    mask_image.save(os.path.join(output_masks_dir, image_name.replace('imagery', 'labels')), format='PNG')

# Load and resize sample mask image
sample_image_path = os.path.join(output_masks_dir, '33 OD - Copy.jpeg')
sample_image = cv2.imread(sample_image_path)
sample_image_resized = cv2.resize(sample_image, (640, 640), interpolation=cv2.INTER_NEAREST)

# Label encoding for the mask
n, h, w = sample_image_resized.shape
mask_reshaped = sample_image_resized.reshape(-1, 1)
encoded_labels = LabelEncoder().fit_transform(mask_reshaped)
encoded_labels_reshaped = encoded_labels.reshape(n, h, w)
print(np.unique(encoded_labels_reshaped))

# Store all processed masks in an array
masks_collection = []
for mask_file in glob.glob(os.path.join(output_masks_dir, "*.jpeg")):
    mask = cv2.imread(mask_file, 0)
    resized_mask = cv2.resize(mask, (640, 640), interpolation=cv2.INTER_NEAREST)
    masks_collection.append(resized_mask)

masks_collection = np.array(masks_collection)

# Verify unique labels for sample masks
for idx in range(10):
    print(f"Label {idx} unique values: {np.unique(masks_collection[idx])}")

# Visualize masks
plt.imshow(masks_collection[1])
plt.show()
plt.imshow(masks_collection[3])
plt.show()
